In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
df_X = diabetes.data
df_y = diabetes.target

In [ ]:
import numpy as np

X = np.array(df_X)
y = np.array(df_y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
def model(X, W, b):
    predictions = 0
    for i in range(len(W)):
        predictions += X[:, i] * W[i]
    predictions += b
    return predictions

In [ ]:
def MSE(a, b):
    mse = ((a - b) ** 2).mean()  # 두 값의 차이의 제곱의 평균
    return mse

def loss(X, W, b, y):
    predictions = model(X, W, b)
    L = MSE(predictions, y)
    return L

In [ ]:
def gradient(X, W, b, y):
    N = len(y)
    y_pred = model(X, W, b)
    
    dW = 1/N * 2 * X.T.dot(y_pred - y)
    db = 2 * (y_pred - y).mean()
    return dW, db

## Test
loss의 수렴 정도를 실험하기 위해서 조절해 본 hyperparameters
- Learning rate : 1. 0.01, 0.0001
- EPOCHS : 100, 1000, 10000

In [ ]:
LEARNING_RATES = [1, 0.01, 0.001]
EPOCHS = [100, 1000, 10000]

losses_set = {}
params_set = {}

for i in range(len(EPOCHS)):
    for j in range(len(LEARNING_RATES)):
        # W, b 초기화
        W = np.zeros(X_train.shape[1])
        b = np.random.rand()
        
        # 개별 학습 손실 추적
        losses = []
        
        for k in range(1, EPOCHS[i] + 1):  # EPOCHS[i] 반복
            dW, db = gradient(X_train, W, b, y_train)
            W -= LEARNING_RATES[j] * dW
            b -= LEARNING_RATES[j] * db
            L = loss(X_train, W, b, y_train)
            losses.append(L)
            
            if k % 1000 == 0:
                print(f"EPOCH {EPOCHS[i]}, LR {LEARNING_RATES[j]} - Iteration {k}: Loss {L:.4f}")

        # 최종 손실 저장
        losses_set[f"EPOCHS: {EPOCHS[i]}, LEARNING: {LEARNING_RATES[j]}"] = losses
        # 최종 파라미터 저장
        params_set[f"EPOCHS: {EPOCHS[i]}, LEARNING: {LEARNING_RATES[j]}"] = (W, b)

print("Finish training...")

In [ ]:
print(losses_set.keys())
print(params_set)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
for key, losses in losses_set.items():
    plt.plot(losses, label=key)

plt.title("Losses Over Iterations for Different Hyperparameters")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
    
for key, (W, b) in params_set.items():
    prediction = model(X_test, W, b)  # 예측값 계산
    plt.scatter(X_test[:, 0], prediction, label=f"Prediction ({key})", alpha=0.6)

# 실제값 추가
plt.scatter(X_test[:, 0], y_test, color='blue', label='Actual', marker='x')

# 그래프 설정
plt.title("Actual vs Predicted Values for Different Hyperparameters")
plt.xlabel("Feature: X_test[:, 0]")
plt.ylabel("Target")
plt.legend()
plt.grid(True)
plt.show()